# Лаб-3. Рекомендательные системы

In [13]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd

In [ ]:
# Выбираем девайс
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Device: {device}')

В качестве датасета будем использовать MovieLens

https://grouplens.org/datasets/movielens/

А именно, самый маленький вариант со 100 тыс. оценок

https://files.grouplens.org/datasets/movielens/ml-latest-small.zip


In [15]:
# Для загрузки датасета напишем свою реализацию класса Dataset
class MovielensDataset(Dataset):
    r"""seed должен быть одинаковым для обучающей и тренировочной выборки"""
    def __init__(self, source, train=True, seed=1):
        ratings      = pd.read_csv(rf"{source}\ratings.csv")
        self.movies  = pd.read_csv(rf"{source}\movies.csv")

        # Преобразовываем Id фильмов в индексы в таблице movies
        x = self.movies.loc[:,['movieId']]
        x['movieId'], x.index = x.index, x['movieId'].values
        ratings['movieId'] = ratings['movieId'].map(x.to_dict()['movieId'])

        # делим датасет 80% на 20%
        train_data = ratings.sample(frac=0.8, random_state=seed)
        test_data  = ratings.drop(train_data.index)

        self.ratings = train_data if train else test_data

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        sample = self.ratings.iloc[idx]
        return {
            "user": torch.LongTensor([sample['userId']]),
            "movie": torch.LongTensor([sample['movieId']]),
            "rating": torch.FloatTensor([sample['rating']])
        }


batch_size = 200

sataset_source = r'.\ml-latest-small'

movielens_train = MovielensDataset(sataset_source, train=True)
movielens_test  = MovielensDataset(sataset_source, train=False)

train_loader = DataLoader(movielens_train, batch_size, True)
test_loader = DataLoader(movielens_test, batch_size, True)

In [ ]:
for batch in train_loader:
    for k, v in batch.items():
        print(k, v.shape)
    break

In [20]:
# Функции для обучения из прошлой лабы, с учётом юзеров и айтемов

def train_iteration(model, data_loader, loss_function, optimizer):
    model.train()
    train_size = len(data_loader.dataset)
    for idx, batch in enumerate(data_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        pred = model(batch)
        loss = loss_function(pred, batch['rating'])
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if idx % 100 == 0:
            loss, current = loss.item(), (idx + 1) * batch_size
            print(f"loss: {loss:>7f}  [{current:>5d}/{train_size:>5d}]")

def test(model, data_loader, loss_function):
    model.eval()
    num_batches = len(data_loader)
    loss = 0
    with torch.no_grad():
        for batch in data_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            pred = model(batch)
            loss += loss_function(pred, batch['rating']).item()

    loss /= num_batches
    print(f"Avg loss: {loss:>8f} \n")


def train(epochs, model, loss_function, optimizer):
    for t in range(epochs):
        print(f"== Epoch {t + 1} ==")
        train_iteration(model, train_loader, loss_function, optimizer)
        test(model, test_loader, loss_function)


## Матричные разложения

В матричных разложениях используется таблица юзеров-айтемов -- таблица, где по строкам находятся юзеры, по столбцам айтемы, на пересечениях оценка, которую поставил пользователь.

Эта таблица представляется в виде произведения двух матриц, матрицы пользователей и матрицы айтемов

![разложение](images/PQ.drawio.png)

В каждом столбце матрицы пользователей живёт вектор, соответствующий этому пользователю, в матрице айтема, соответственно, вектор айтема. Чтобы получить предсказание оценки, надо их перемножить.

Есть много разных способов находить матричные разложения, поскольку у нас тут pytorch, мы просто возьмём два `Embedding` слоя, перемножим, и скажем что это наша модель, которую обучим градиентным спуском


In [ ]:
class MatrixFactorization(nn.Module):
    def __init__(self):
        super().__init__()
        self.user_embeddings  = nn.Embedding(1000,  16)
        self.movie_embeddings = nn.Embedding(10000, 16)

    def forward(self, batch):
        movie_emb = self.user_embeddings(batch['user'])
        user_emb = self.movie_embeddings(batch['movie'])
        return (movie_emb * user_emb).sum(2)


mf_model = MatrixFactorization().to(device)
mf_loss = nn.MSELoss()
mf_optimizer = torch.optim.SGD(mf_model.parameters(), lr=1)

train(10, mf_model, mf_loss, mf_optimizer)

Фактически, если к этой моделе в сумму добавить общую константу и константу для кадого пользователя и айтема, мы получим Factorization Machine

https://www.ismll.uni-hildesheim.de/pub/pdfs/Rendle2010FM.pdf

А это значит, что помимо эмбедингов с юзерами и айтемами мы можем легко добавить дополнительных параметров! (например тех, что у нас в таблице tags.csv)

## DeepFM

DeepFM это расширение обычной Factorization Machine для использования 

https://arxiv.org/pdf/1703.04247

Идея состоит в том, чтобы расположить рядом с обычной Factorization Machine нейронную сеть, которая будет параллельно существовать с матричным разложением

![DeepFM](images/DeepFM.png)

До DeepFM уже были модели, которые предварительно обучали эмбединги на матричном разложении, а потом использовали их как входные векторы сети, но тут предлагается обучать их сразу совместно

In [ ]:
class DeepFM(nn.Module):
    def __init__(self):
        super().__init__()
        self.user_embeddings  = nn.Embedding(1000,  16)
        self.movie_embeddings = nn.Embedding(10000, 16)

        self.flatten = nn.Flatten()

        self.deep_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
        )

        self.final_layer = nn.Linear(16*3, 1)

    def forward(self, batch):
        movie_emb = self.flatten(self.user_embeddings(batch['user']))
        user_emb  = self.flatten(self.movie_embeddings(batch['movie']))

        fm = movie_emb * user_emb

        deep = torch.cat([movie_emb, user_emb], 1)
        deep = self.deep_layers(deep)

        v = torch.cat([fm, deep], 1)
        v = self.final_layer(v)
        # делаем сигмоиду на выходе и масштабируем к оценкам от 0 до 5
        return torch.sigmoid(v) * 5


deep_mf_model = DeepFM().to(device)
deep_mf_loss = nn.MSELoss()
deep_mf_optimizer = torch.optim.SGD(deep_mf_model.parameters(), lr=1e-1)

train(5, deep_mf_model, deep_mf_loss, deep_mf_optimizer)

Есть и более прокаченные версии машины факторизации на нейронках, например xDeepFM

https://arxiv.org/pdf/1803.05170

## Задание

Основное задание:
1) Достичь меньше чем 0.8 значения MSELoss на этом датасете (5 баллов)
2) МОЖНО ДЕЛАТЬ ТОЛЬКО ПОСЛЕ ТОГО КАК СДЕЛАНО ПЕРВОЕ ЗАДАНИЕ!  
    Добавить в тренировочный датасет нового пользователя - себя и дать оценки минимум 20 фильмов, обучить модель с учётом этого пользователя и сделать для себя рекомендации. (5 баллов) (пожалуйста, не дописывайте себя в файлик, сделайте пользователя добавление в питоне)

Дополнительные задания:
1) Добавить в модель использование тегов из таблички `tags.csv` (5 дополнительных баллов)
2) Добавить в модель использование дополнительных данных из источников `links.csv` (5 дополнительных баллов)